In [ ]:
import pandas as pd
from google.colab import files, drive

In [ ]:
uploaded = files.upload()
data = pd.read_csv('combined_df_label.csv', index_col=0)

Saving combined_df_label.csv to combined_df_label (3).csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,22,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df = pd.DataFrame(data)

#Removing columns from the dataframe

df.drop(columns=['coordinates', 'created_at',	'hashtags',	'media',	'urls',	'favorite_count',	'id',	'in_reply_to_screen_name',	'in_reply_to_status_id',	'in_reply_to_user_id',	'lang',	'place',	'possibly_sensitive',	'retweet_count',	'retweet_id',	'retweet_screen_name',	'source',	'tweet_url',	'user_created_at', 'user_screen_name',	'user_default_profile_image',	'user_description',	'user_favourites_count',	'user_followers_count', 'user_friends_count',	'user_listed_count',	'user_location',	'user_name',	'user_screen_name.1',	'user_statuses_count',	'user_time_zone',	'user_urls',	'user_verified'], axis=1)

,text,label
0,What is a coronavirus immunity certificate? US...,real
1,@oluafolabi What is a Coronavirus in the prese...,real
2,What is a coronavirus immunity certificate? US...,real
3,What is a coronavirus immunity certificate? US...,real
4,@AngryDuck91 - Does WHO recommend that all int...,real
...,...,...
214322,"@Yamiche Remember 40,000 new cases/day transla...",real
214323,@jaketapper I was just wondering if all other ...,real
214324,@jaketapper @MalcolmNance Those are damning nu...,real
214325,@jaketapper @MalcolmNance There is no more dam...,real


In [ ]:
df.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified,label
0,NaN,Fri Apr 10 20:58:29 +0000 2020,NaN,NaN,https://trib.al/rXNwtk7,1,1248717006951645190,NaN,NaN,NaN,en,NaN,False,0.0,NaN,NaN,"<a href=""http://www.socialflow.com"" rel=""nofol...",What is a coronavirus immunity certificate? US...,https://twitter.com/FresnoBee/status/124871700...,Fri Aug 17 17:44:58 +0000 2007,FresnoBee,False,The No. 1 source of news in California's centr...,3384.0,93055.0,782.0,1016.0,"Fresno, California",Fresno Bee,FresnoBee,131032.0,NaN,http://www.fresnobee.com,True,real
1,NaN,Sat Apr 25 17:42:12 +0000 2020,NaN,NaN,NaN,0,1254103429313593344,oluafolabi,1.254102e+18,177972516.0,en,NaN,NaN,0.0,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",@oluafolabi What is a Coronavirus in the prese...,https://twitter.com/casualhalle/status/1254103...,Thu Mar 28 17:56:49 +0000 2013,casualhalle,False,Minister of Reconciliation • Design Evangelist...,21026.0,253.0,293.0,0.0,Nigeria,"judas, not iscariot.",casualhalle,3597.0,NaN,NaN,False,real
2,NaN,Fri Apr 10 22:22:47 +0000 2020,NaN,NaN,https://trib.al/1blBJUb,6,1248738224513703936,NaN,NaN,NaN,en,NaN,False,5.0,NaN,NaN,"<a href=""http://www.socialflow.com"" rel=""nofol...",What is a coronavirus immunity certificate? US...,https://twitter.com/KCStar/status/124873822451...,Fri May 30 16:08:18 +0000 2008,KCStar,False,Breaking the news in Kansas City since 1880.,4470.0,292492.0,862.0,2914.0,"Kansas City, Mo.",The Kansas City Star,KCStar,260568.0,NaN,http://www.kansascity.com,True,real
3,NaN,Sun Apr 12 00:27:54 +0000 2020,NaN,NaN,https://www.kansascity.com/news/coronavirus/ar...,0,1249132096993538048,NaN,NaN,NaN,en,NaN,False,0.0,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",What is a coronavirus immunity certificate? US...,https://twitter.com/raiderzgirl1972/status/124...,Wed Mar 07 05:44:38 +0000 2012,raiderzgirl1972,False,NaN,2525.0,98.0,629.0,0.0,St Joseph Mo,Angela Weston,raiderzgirl1972,3361.0,NaN,NaN,False,real
4,NaN,Tue Apr 21 12:38:09 +0000 2020,NaN,NaN,NaN,0,1252577361545043968,NaN,NaN,NaN,en,NaN,NaN,0.0,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@AngryDuck91 - Does WHO recommend that all int...,https://twitter.com/AngelaLuna80/status/125257...,Thu Aug 11 08:44:42 +0000 2011,AngelaLuna80,False,NaN,12782.0,71.0,386.0,0.0,NaN,LUNHERAN,AngelaLuna80,3556.0,NaN,NaN,False,real


In [ ]:
df = df[["text", "label"]]
print(df.shape)
print(df.head())

(214327, 2)
                                                text label
0  What is a coronavirus immunity certificate? US...  real
1  @oluafolabi What is a Coronavirus in the prese...  real
2  What is a coronavirus immunity certificate? US...  real
3  What is a coronavirus immunity certificate? US...  real
4  @AngryDuck91 - Does WHO recommend that all int...  real


In [ ]:
# data cleaning

import re
import nltk
import pandas as pd
from textblob import Word
import numpy as np
from sklearn import preprocessing
import string
from nltk.stem import WordNetLemmatizer


from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('wordnet')

def text_clean (text):
  try:
    lines = re.sub("https?:\/\/.*[\r\n]*", "", text) #removes urls
    lines = re.sub("\$", "", lines) #removes market tickers 
    lines = re.sub("[^\w\s]", "", lines) #removes punctuation
    lines = re.sub("@\S+", "", lines) #removes mentions
    lines = re.sub(r"\([^()]*\)", "", lines) #removes text within parentheses
  except Exception as e:
    print(e)
    print(text)
  return lines.strip().lower() # lowercasing the text

#stopword removal and lemmatilization
x = data['text'].tolist()
y = data['label'].tolist()
for index,value in enumerate(x):
  x[index] = ' '.join([Word(word).lemmatize() for word in text_clean(value).split() if word not in set(stopwords.words('english'))])


print(df.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


KeyboardInterrupt: ignored

In [ ]:
data = {'label': y, 'text': x}
df = pd.DataFrame(data)

# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

# Write the DataFrame to CSV file.
with open('/gdrive/My Drive/datapreprocessed.csv', 'w') as f:
  df.to_csv(f)



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
uploaded = files.upload()
data = pd.read_csv('datapreprocessed.csv', index_col=0)

Saving datapreprocessed.csv to datapreprocessed.csv


In [ ]:
#data = pd.read_csv('/gdrive/My Drive/datapreprocessed.csv', index_col=0)

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in 'label'. 
print(data['label'].unique())
data['label']= label_encoder.fit_transform(data['label']) 
data['label'].unique()

['real' 'fake']


array([1, 0])

In [ ]:
import numpy as np

df1 = data[data.isna().any(axis=1)]

print(df1)
data = data.dropna()


        label text
6           1  NaN
34          1  NaN
1579        1  NaN
3365        1  NaN
3490        1  NaN
...       ...  ...
250791      0  NaN
250793      0  NaN
250859      0  NaN
250889      0  NaN
250894      0  NaN

[561 rows x 2 columns]


In [ ]:
data[:2]

,label,text
0,1,coronavirus immunity certificate u may start i...
1,1,oluafolabi coronavirus presence corrosive anoi...


In [ ]:
#vectorization
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

x = data['text'].tolist()
y = data['label'].tolist()
x_train, x_test = train_test_split(x, test_size=0.30, random_state=1000)
Y_train, Y_test = train_test_split(y, test_size=0.30, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit (x_train)
X_train = vectorizer.transform(x_train)
X_test  = vectorizer.transform(x_test)

Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.layers import Dropout, Dense, GRU, Embedding
from sklearn import metrics
from sklearn import preprocessing 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

model = Sequential()
node = 128 # number of nodes
nLayers = 2 # number of  hidden layer
shape = X_train.shape[1] 
dropout = 0.5
nClasses = 2
model.add(Dense(node,input_dim=shape,activation='relu'))
model.add(Dropout(dropout))
for i in range(0,nLayers):
    model.add(Dense(node,input_dim=node,activation='relu'))
    model.add(Dropout(dropout))
model.add(Dense(nClasses, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:

history = model.fit(X_train, Y_train,
                          validation_data=(X_test, Y_test),
                          epochs=8,
                          batch_size=32,
                          verbose=2)

Epoch 1/8


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_1/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_1/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_1/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


5511/5511 - 1367s - loss: 0.0961 - accuracy: 0.9755 - val_loss: 0.0664 - val_accuracy: 0.9814
Epoch 2/8
5511/5511 - 1363s - loss: 0.0428 - accuracy: 0.9885 - val_loss: 0.0616 - val_accuracy: 0.9845
Epoch 3/8
5511/5511 - 1368s - loss: 0.0238 - accuracy: 0.9936 - val_loss: 0.0714 - val_accuracy: 0.9845
Epoch 4/8
5511/5511 - 1390s - loss: 0.0163 - accuracy: 0.9961 - val_loss: 0.0804 - val_accuracy: 0.9849
Epoch 5/8
5511/5511 - 1369s - loss: 0.0115 - accuracy: 0.9975 - val_loss: 0.1066 - val_accuracy: 0.9845
Epoch 6/8
5511/5511 - 1364s - loss: 0.0103 - accuracy: 0.9977 - val_loss: 0.0943 - val_accuracy: 0.9846
Epoch 7/8
5511/5511 - 1368s - loss: 0.0087 - accuracy: 0.9983 - val_loss: 0.1101 - val_accuracy: 0.9847
Epoch 8/8
5511/5511 - 1371s - loss: 0.0093 - accuracy: 0.9984 - val_loss: 0.1092 - val_accuracy: 0.9850


In [ ]:
loss, accuracy = model.evaluate(X_train, Y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
predictions = model.predict(X_test)
matrix = metrics.confusion_matrix(Y_test, predictions.argmax(axis=1))
print(matrix)

Training Accuracy: 0.9997
Testing Accuracy:  0.9850
[[ 2972   903]
 [  233 71466]]


In [ ]:
model.summary()
predicted = model.predict(X_test)
print(metrics.classification_report (Y_test, np.argmax(predicted, axis = 1)))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               22091392  
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                